In [1]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
import cv2
import scipy
import os
from keras.preprocessing import image
from keras.models import Model
from keras.layers.pooling import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.python.client import device_lib
from keras import optimizers
import pickle
from six.moves import cPickle
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, \
    Convolution2D, MaxPooling2D, ZeroPadding2D, Input, Embedding, LSTM, merge, \
    Lambda, UpSampling2D, Deconvolution2D, Cropping2D
import os
import glob
import numpy as np
import keras
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv2DTranspose, Activation, concatenate, Reshape, Permute
from keras.optimizers import Adam
from keras import losses
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras import backend as K
import cv2
import tensorflow as tf
from scipy.misc import toimage
import itertools
import h5py
import skimage
import skimage.transform

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
K.set_image_dim_ordering('tf')

In [3]:
def preprocess(imgs, imgNorm):
    for i in range(len(imgs)):
        img = imgs[i]
        if imgNorm == "sub_and_divide":
            img = img.astype(np.float32)
            img = img / 127.5 - 1
        elif imgNorm == "sub_mean":
            img = img.astype(np.float32)
            img[:,:,0] -= 103.939
            img[:,:,1] -= 116.779
            img[:,:,2] -= 123.68
        elif imgNorm == "divide":
            img = img.astype(np.float32)
            img = img/255.0
    return imgs


In [4]:
def getSegmentationArr( path , nClasses ,  width , height  ):

    seg_labels = np.zeros((  height , width  , nClasses ))
    try:
        img = cv2.imread(path, 1)
        img = cv2.resize(img, ( width , height ))
        img = img[:, : , 0]

        for c in range(nClasses):
            seg_labels[: , : , c ] = (img == c ).astype(int)

    except Exception as e:
        print(e)

    seg_labels = np.reshape(seg_labels, ( width*height , nClasses ))
    return seg_labels

In [5]:
def imageSegmentationGenerator( images_path , segs_path ,  batch_size,  n_classes , input_height , input_width , output_height , output_width   ):
    
    assert images_path[-1] == '/'
    assert segs_path[-1] == '/'

    images = glob.glob( images_path + "*.jpg"  ) + glob.glob( images_path + "*.png"  ) +  glob.glob( images_path + "*.jpeg"  )
    images.sort()
    segmentations  = glob.glob( segs_path + "*.jpg"  ) + glob.glob( segs_path + "*.png"  ) +  glob.glob( segs_path + "*.jpeg"  )
    segmentations.sort()

    assert len( images ) == len(segmentations)
    for im , seg in zip(images,segmentations):
        assert(  im.split('/')[-1].split(".")[0] ==  seg.split('/')[-1].split(".")[0] )

    zipped = itertools.cycle( zip(images,segmentations) )

    while True:
        X = []
        Y = []
        for _ in range( batch_size) :
            im , seg = zipped.next()
            X.append( getImageArr(im , input_width , input_height )  )
            Y.append( getSegmentationArr( seg , n_classes , output_width , output_height )  )

        yield np.array(X) , np.array(Y)

In [6]:
def new_cross_entropy(y_, y_conv):
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
    return cross_entropy

In [7]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [8]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [9]:
def dice(y_true, y_pred):
    y_true_f = K.clip(K.batch_flatten(y_true), 0., 1.)
    y_pred_f = K.clip(K.batch_flatten(y_pred), 0., 1.)

    intersection = 2 * K.sum(y_true_f * y_pred_f, axis=1)
    union = K.sum(y_true_f * y_true_f, axis=1) + K.sum(y_pred_f * y_pred_f, axis=1)
    return K.mean(intersection / union)

In [10]:
def dice_loss(y_true, y_pred):
    return -dice(y_true, y_pred)

In [11]:
def log_dice_loss(y_true, y_pred):
    return -K.log(dice(y_true, y_pred))

In [12]:
def dice_metric(y_true, y_pred):
    """An exact Dice score for binary tensors."""
    y_true_f = K.greater(y_true, 0.5)
    y_pred_f = K.greater(y_pred, 0.5)
    return dice(y_true_f, y_pred_f)

In [13]:
def preprocess_other(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, 3), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        arr = imgs[i]
        arr = arr.astype('float32')
        arr /= 255.
        imgs_p[i] = resize(arr, (img_cols, img_rows), preserve_range=True)
    return imgs_p

In [14]:
def resize_multi(arr, rows, cols, channels):
    for i in range(len(arr)):
        arr[i] = skimage.transform.resize(arr[i], (rows, cols, channels))
    return arr

In [15]:
def create_list(arr):
    temp = []
    for image in arr:
        temp.append(image)
    return temp

In [16]:
def get_data(numpy_arr, glauc_bool):
    X = []
    y = []

    for image in numpy_arr:
            X.append(image)
            y.append(glauc_bool)
    X = np.array(X)
    y = np.array(y)
    return X,y



In [17]:
def multiple_image_numpy(imgs, rows, cols, channels):
    total = len(imgs)
    new = np.ndarray((total, rows, cols, channels), dtype=np.uint8)

    for i in range(total):
        new[i] = imgs[i]
    
    return new


In [18]:
def sort(path, im_type, backend):

    images = glob.glob1(path,"*." + str(im_type))
    im_type_length = len(im_type) + 1
    images = [element[:-im_type_length] for element in images]
    images.sort(key=int)
    images = [element + "." + str(im_type) for element in images]

    if backend == "keras" or backend == "Keras" or backend == "keras":
        images = [np.array(image.load_img(path + "/" + fname)) for fname in images]
    if backend == "opencv" or backend == "openCV" or backend == "OPENCV" or backend == "cv2":
        images = [np.array(cv2.imread(path + "/" + fname)) for fname in images]
    if backend == "PIL" or backend == "pil":
        images = [np.array(Image.open(path + "/" + fname)) for fname in images]
    return images

In [19]:
# def get_inter_model():
#     model_vgg19_conv = VGG19(weights='imagenet', include_top=False, input_shape=(None,None,3))

#     conv2_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv2_3_16", data_format="channels_last")(model_vgg19_conv.layers[-17].output)

#     conv3_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv3_3_16", data_format="channels_last")(model_vgg19_conv.layers[-12].output)

#     conv4_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv4_3_16", data_format="channels_last")(model_vgg19_conv.layers[-7].output)

#     conv5_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv5_3_16", data_format="channels_last")(model_vgg19_conv.layers[-2].output)

#     upsample2_ = Conv2DTranspose(filters=16, kernel_size=(4, 4), strides=(2, 2), padding = "same", name= "upsample2_", data_format="channels_last")(conv2_3_16)
    
#     upsample4_ = Conv2DTranspose(filters=16, kernel_size=(8, 8), strides=(4, 4), padding = "same", name= "upsample4_", data_format="channels_last")(conv3_3_16)

#     upsample8_ = Conv2DTranspose(filters=16, kernel_size=(16, 16), strides=(8, 8), padding = "same",  name= "upsample8_", data_format="channels_last")(conv4_3_16)

#     upsample16_ = Conv2DTranspose(filters=16, kernel_size=(32, 32), strides=(16, 16), padding = "same", name= "upsample16_", data_format="channels_last")(conv5_3_16)

#     concat_upscore = concatenate([upsample2_, upsample4_, upsample8_, upsample16_], axis=-1)
    
#     """
#     new_score_weighting = Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid', kernel_initializer=keras.initializers.Constant(value=0.01), name= "new_score_weighting", data_format="channels_last")(concat_upscore)
    
#     my_model = Model(input=model_vgg19_conv.input , output=new_score_weighting)
    
#     """

#     conv6 = Conv2D(filters=1, kernel_size=(1,1), activation='relu', kernel_initializer=keras.initializers.Constant(value=0.01), name= "conv6", data_format="channels_last", border_mode='same')(concat_upscore)
#     conv6 = Reshape((1,input_shape[0]*input_shape[1]))(conv6)
#     conv6 = Permute((2,1))(conv6)

#     conv7 = Activation('softmax')(conv6)

#     my_model = Model(input=model_vgg19_conv.input, output=conv7)

#     return my_model

In [20]:
def get_classifier_model():
    model_vgg19_conv = VGG19(weights='imagenet', include_top=False, input_shape=(128,128,3))

    conv2_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv2_3_16", data_format="channels_last")(model_vgg19_conv.layers[-17].output)

    conv3_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv3_3_16", data_format="channels_last")(model_vgg19_conv.layers[-12].output)

    conv4_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv4_3_16", data_format="channels_last")(model_vgg19_conv.layers[-7].output)

    conv5_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv5_3_16", data_format="channels_last")(model_vgg19_conv.layers[-2].output)

    upsample2_ = Conv2DTranspose(filters=16, kernel_size=(4, 4), strides=(2, 2), padding = "same", name= "upsample2_", data_format="channels_last")(conv2_3_16)
    
    upsample4_ = Conv2DTranspose(filters=16, kernel_size=(8, 8), strides=(4, 4), padding = "same", name= "upsample4_", data_format="channels_last")(conv3_3_16)

    upsample8_ = Conv2DTranspose(filters=16, kernel_size=(16, 16), strides=(8, 8), padding = "same",  name= "upsample8_", data_format="channels_last")(conv4_3_16)

    upsample16_ = Conv2DTranspose(filters=16, kernel_size=(32, 32), strides=(16, 16), padding = "same", name= "upsample16_", data_format="channels_last")(conv5_3_16)

    concat_upscore = concatenate([upsample2_, upsample4_, upsample8_, upsample16_], axis=-1)
    
    conv6 = Conv2D(filters=1, kernel_size=(1,1), activation='relu', kernel_initializer=keras.initializers.Constant(value=0.01), name= "conv6", data_format="channels_last", border_mode='same')(concat_upscore)

    
    conv_1 = Conv2D(filters=32, kernel_size=(3, 3), name= "conv_1", activation = 'relu', data_format="channels_last")(conv6)
    max_pool1 = MaxPooling2D(pool_size=(2, 2), name= "max_pool1", data_format="channels_last")(conv_1)
    
    conv_2 = Conv2D(filters=32, kernel_size=(3, 3), name= "conv_2", activation = 'relu', data_format="channels_last")(max_pool1)
    max_pool2 = MaxPooling2D(pool_size=(2, 2), name= "max_pool2", data_format="channels_last")(conv_2)

    conv_3 = Conv2D(filters=64, kernel_size=(3, 3), name= "conv_3", activation = 'relu', data_format="channels_last")(max_pool2)
    max_pool3= MaxPooling2D(pool_size=(2, 2), name= "max_pool3", data_format="channels_last")(conv_3)

    flatten = Flatten(input_shape = model_vgg19_conv.input.shape)(max_pool3)
    dense_1 = Dense(16, activation='relu')(flatten)
    dropout = Dropout(0.7)(dense_1)
    dense_2 = Dense(1, activation='sigmoid')(dropout)

    my_model = Model(input=model_vgg19_conv.input, output=dense_2)
    
    my_model.summary()

    return my_model

In [21]:
classifier_model = get_classifier_model()
classifier_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_initializer=<keras.ini..., name="conv6", activation="relu", data_format="channels_last", padding="same", filters=1, kernel_size=(1, 1))`
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 128, 128, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 128, 128, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 64, 64, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [32]:
glauc = sort("/Users/karimabedrabbo/Desktop/work2/cropped_glauc_orig_png", "png", "keras")
not_glauc = sort("/Users/karimabedrabbo/Desktop/work2/cropped_not_glauc_orig_png", "png", "keras")

In [33]:
y_glauc = np.ones(len(glauc))
y_not_glauc = np.zeros(len(not_glauc))

In [34]:
glauc = resize_multi(glauc, 128, 128, 3)
not_glauc = resize_multi(not_glauc, 128, 128, 3)

In [35]:
glauc = multiple_image_numpy(glauc, 128, 128, 3)
not_glauc = multiple_image_numpy(not_glauc, 128, 128, 3)

In [36]:
glauc = preprocess(glauc, "divide")
not_glauc = preprocess(not_glauc, "divide")

In [37]:
x_final = np.vstack((glauc, not_glauc))
y_final = np.hstack((y_glauc, y_not_glauc))

In [38]:

model_checkpoint = ModelCheckpoint('karim_best_weights_new.h5', monitor='val_loss', save_best_only=True)

#model.fit_generator(generator=cup_train_data_gen, steps_per_epoch=104, epochs=40, verbose=1, callbacks=[model_checkpoint], validation_data=cup_val_data_gen, validation_steps=26, shuffle=True)

classifier_model.fit(x=np.array(x_final), y=np.array(y_final), batch_size=16, epochs=1000, verbose=1, callbacks=[model_checkpoint], validation_split=0.2, shuffle=True)


Train on 1240 samples, validate on 311 samples
Epoch 1/1000
  32/1240 [..............................] - ETA: 22:14 - loss: 3.8858 - acc: 0.5312

KeyboardInterrupt: 